In [32]:
!pip install mlflow torch evaluate transformers --quiet

In [33]:
!pip install pyngrok

In [34]:
get_ipython().system_raw("mlflow ui --port 5000 &")


In [35]:
from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
ngrok.kill()


In [36]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


In [37]:
import transformers

import mlflow
from typing import List, Dict

# Define the task that we want to use (required for proper pipeline construction)
task = "feature-extraction"

# Define the pipeline, using the task and a model instance that is applicable for our task.
generation_pipeline = transformers.pipeline(
    task=task,
    model="sergeyzh/rubert-mini-sts",
    model_kwargs= {"force_download":True, "is_decoder":True}

)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/130M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

In [38]:
type(generation_pipeline)

transformers.pipelines.feature_extraction.FeatureExtractionPipeline

In [39]:
input_example : str = "prompt 1"

# Define the parameters (and their defaults) for optional overrides at inference time.
parameters = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.4}

In [40]:
type(input_example)

str

In [41]:
signature = mlflow.models.infer_signature(
    input_example,
    mlflow.transformers.generate_signature_output(generation_pipeline, input_example),
    parameters,
)

In [42]:
signature

inputs: 
  [string (required)]
outputs: 
  [Tensor('float64', (-1,))]
params: 
  ['max_new_tokens': long (default: 512), 'do_sample': boolean (default: True), 'temperature': double (default: 0.4)]

In [43]:
mlflow.set_tracking_uri("http://localhost:5000")


experiment_name = "Embeddings_experiment"
try:
    mlflow.create_experiment(experiment_name)
except:
    print("experiment exists")
mlflow.set_experiment(experiment_name)

experiment exists


<Experiment: artifact_location='mlflow-artifacts:/689960186442330333', creation_time=1718206500139, experiment_id='689960186442330333', last_update_time=1718206500139, lifecycle_stage='active', name='Embeddings_experiment', tags={}>

Save Model into **MLflow**

In [45]:
from datetime import datetime
import pandas as pd
## Save Model into MLflow
name = experiment_name +datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
with mlflow.start_run(run_name = name) as run:
    model_info = mlflow.transformers.log_model(
        transformers_model=generation_pipeline,
        artifact_path="text_generator",
        input_example=input_example,
        signature=signature,
        # Uncomment the following line to save the model in 'reference-only' mode:
        # save_pretrained=False,
    )

<ipython-input-45-5b7d08ea80f3>:6: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.41.0``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  model_info = mlflow.transformers.log_model(
/usr/local/lib/python3.10/dist-packages/mlflow/models/model.py:666: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.41.0``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2024/06/12 16:01:25 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.0+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.3.0' without the local version label 

In [46]:
model_info.model_uri


'runs:/3087d1ee4f6d4963ad56bc2e02c8b249/text_generator'

In [47]:
text_generator = mlflow.pyfunc.load_model(model_info.model_uri)


2024/06/12 16:01:48 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


In [48]:
type(text_generator)


mlflow.pyfunc.PyFuncModel

In [49]:
predictions = text_generator.predict(
    data=[
        "I can't decide whether to go running or kayaking this weekend. Can you help me decide?",
        "Please tell me a joke about chess.",
    ],
    params={"temperature": 0.9},
)

2024/06/12 16:01:54 WARNING mlflow.transformers: params provided to the `predict` method will override the inference configuration saved with the model. If the params provided are not valid for the pipeline, MlflowException will be raised.


In [52]:
predictions = text_generator.predict(
    data=
        "I can't decide whether to go running or kayaking this weekend. Can you help me decide?"
)

2024/06/12 16:02:18 WARNING mlflow.transformers: params provided to the `predict` method will override the inference configuration saved with the model. If the params provided are not valid for the pipeline, MlflowException will be raised.


In [53]:
predictions.shape

(24, 312)

In [56]:
predictions[1]


array([-2.06378746e+00,  1.65152848e-01, -4.78393972e-01, -2.68588448e-03,
       -3.65058720e-01,  6.92861736e-01,  6.99380338e-01,  5.36932237e-02,
       -2.58363515e-01,  7.20451176e-01,  5.92592955e-01, -6.12084925e-01,
       -1.27576447e+00, -2.06943408e-01, -2.98365146e-01,  2.28391320e-01,
        8.39091837e-01,  8.95106256e-01, -2.43978873e-01, -6.27535164e-01,
        5.04434526e-01,  3.63060147e-01,  3.59200895e-01, -4.18931842e-01,
       -3.91544193e-01,  3.22141987e-03,  1.79114208e-01,  4.37342525e-01,
        1.20686150e+00,  3.53774071e-01,  3.11460316e-01, -5.74246168e-01,
        1.34204638e+00,  1.66360453e-01, -3.28754038e-01,  8.78118932e-01,
        1.02786779e+00,  1.29836690e+00, -1.79326892e+00, -1.59887958e+00,
       -9.70536292e-01, -3.89293760e-01,  1.58112574e+00,  1.65084437e-01,
       -4.11094427e-02, -2.60015905e-01,  1.01851988e+00,  1.41122627e+00,
        1.22208071e+00, -7.26689696e-01,  2.20769095e+00, -7.02293575e-01,
        1.06334519e+00,  

In [ ]:
# tasks.py

#pip install torch transformers numpy pandas
import torch
from transformers import AutoTokenizer, AutoModel
from numpy import dot
from numpy.linalg import norm

import pandas


#from celery_app import app

#conn = Redis(host='redis', port=6379)
embedding_model = "sergeyzh/rubert-mini-sts"

tokenizer = AutoTokenizer.from_pretrained(embedding_model)
model = AutoModel.from_pretrained(embedding_model)

# Функция для получения векторного представления текста
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()


embed_bert_cls("I can't decide whether to go running or kayaking this weekend. Can you help me decide?", model, tokenizer).shape

